In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [30]:
import requests
from bs4 import BeautifulSoup


login_url = "https://telemetr.me/login/"  

login_data = {
    'login[email]': 'Klovak.top@yandex.ru',  
    'login[password]': 'Klovak.top@yandex.ru', 
    'recaptcha_response': '03AFcWeA5Qtim3ErT75AJu-7a8TbdsuCMZrl7vKvypPZYrQTxEk3k9nQkb3HGXrkKHb77RKU5LOybYdybw2eXdqJNczDHsXqKqixoKJEeOpOd-qldx1dj-9-ywWpEqe2rSdjjAe83d9qZwsyQdph58Cu_iTvp6pwqbQ3-1F5X0tAdW552EOPveIpOFnZeeq6mgicH9yCVbK5IqJZnpdhM0ub0o5FiwovLEgDAl6q1Fu3WqdWHD235Kmf8vNGMF1HaEtwUiFH2KyJEp0o83J3KsCzU4B8OGcycAsNwOR47JS-ls1N9Lu4cgwCdUUayOpOKV4no_7vndNzJ8W1vuic09rSLMnd20kaMDeP2QcB-RPkrqJ256Ze3WsDK4x9YFOKrBs4I_0AGlbGprfZmfZ9q-KrjIlnFEDYPEZWuzRjOCdESwYewTqTIPiBCTTXfa25xOY0B6EishrJzMyngEa3-ipKzA7q-n6zPq-dR4SYAEHmYTmz1bmYEdOJIjQlbtanm6X_nCJMGA9Xpxb5Ruf-3W_OQNuzF-9IzvFGlxWwzYp3BM4iVUoL5FQEBFwl06SN-LgPmO1uxC2Xq5tTEhISVhm_3_BCmBsiNCiLWmrHjm8Y5WU36p4DOIJ8gsCo2h9wVRfO-akqkHMUWW1xUJjmfWoEUSqXcHjlujqWp5rn-bSTfzbEpSo8n-Of6QDZBKvLOg0VM7VSP_wRkKMsvpDpOHI8MSpfcPSQSsKZBcWHsNUj2iFOB-LxnxkmOfU2D4DnuYzB4sh_xlAt-HZTJUp-Ec88xSoxfbEPKC3ff9u3F5cXI7sDmpP1U4jkenEM7uITRL4yAP0LTo1jLv',
    'do_login': ''
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://telemetr.me/login/',
}

session = requests.Session()
response = session.post(login_url, data=login_data, headers=headers)

from_tg_url = input()
response_first = session.get(from_tg_url)
soup = BeautifulSoup(response_first.text, 'html.parser')


if response_first.ok:
    print("Авторизация успешна!")
else:
    print("Ошибка авторизации.")



 https://telemetr.me/profile/confirm/?confirm=79bef9619bca7dc67a999eee9f8a665d


Авторизация успешна!


In [51]:
def get_tg_admins(min_sub, max_sub, pages=1,category = 'Юмор', format_back = 'null'):
    base_url = 'https://telemetr.me/'
    res=[]
    
    for p in range(1,pages + 1):
        response = session.get(f'{base_url}channels/cat/{category}/?order_column=participants_count&order_direction=DESC&page={p}&participants_from={min_sub}&participants_to={max_sub}&lang_ru=1&lang_uz=0&channel_type=opened&moderate=all&verified=all&detailed_bot_added=all&lang_code=ru')
        #'https://telemetr.me/channels/cat/Юмор/?about=%27%40%27&participants_from=300&participants_to=3000&channel_type=opened&lang_code=ru&detailed_bot_added=all'
        soup = BeautifulSoup(response.text)
        tables = soup.find_all('tr', class_ = ['tr_even','tr_odd'])

        for table in tables:
            #time.sleep(0.2)
            
            try:
                title = table.find('a', 'kt-ch-title').text
            except:
                continue
                
            try:
                subscribers = table.find('span', 'kt-number kt-font-brand text-cursor').text            
                subscribers = re.search('\d.\d{3}', subscribers).group().replace("'", "")
            except:
                continue 
                
            
            channel_link = base_url + table.find('a', 'kt-ch-title').get('href')[1:]
            channel_info = session.get(channel_link)
            channel_info.encoding = 'utf-8' 
            soup_info = BeautifulSoup(channel_info.text)
            
            try:
                a = soup_info.find('div', 'kt-widget__desc t_long').find_all('a')
                admins = []
                for link in a:
                    admin = link.get('href')
                    if re.search('^https://t\.me/(?!\+)(?!.*[bB][oO][tT]$).*', admin):
                        admins.append(admin)
                if len(admins) == 0:
                    continue
            except:
                continue
                
            try:
                link = soup_info.find('a', 'kt-widget__username').get('href')
                #link = re.search('@\w*', link).group()
            except:
                continue    
            
            
            try:
                description = soup_info.find('div', 'kt-widget__desc t_long').text 
                description = re.sub(r'^\s+', '', description)
            except:
                continue 
                        

            sub_month = soup_info.find('div', id='new_week').find('span', class_ = ['kt-number kt-font-success', 'kt-number kt-font-danger']).text 
            
            
                
            res.append({'title': title, 'link': link, 'subscribers': subscribers, 'sub_month': sub_month,'description': description, 'admins': '\n'.join(admins)})
            
    res_df = pd.DataFrame(res)
    
    if format_back == 'csv':
        res_df.to_csv('results.csv', index=False)
    elif format_back == 'excel':
        res_df.to_excel('results.xlsx', sheet_name='Admins List', index=False)
        
    return res_df


In [50]:
result = get_tg_admins(300,3000,pages = 5,format_back = 'excel')
result.head()

,title,link,subscribers,sub_month,description,admins
0,загадочные обстоятельства,https://t.me/zobst,2930,-60,"\n\t\t\t\t\t\t\t\t\t\t\t\tСтрашно, очень страш...",https://t.me/allldavis\nhttps://t.me/merlegraf
1,АКАДЕМИЯ СМЕХА ?,https://t.me/academylaughtoff,2901,-57,"\n\t\t\t\t\t\t\t\t\t\t\t\tУгарные мемы 😂 ,треш...",https://t.me/academytg1
2,IQ boost ?,https://t.me/iq_boost,2894,-37,\n\t\t\t\t\t\t\t\t\t\t\t\t🧩 Solve puzzles and ...,https://t.me/vikatom
3,нетолерантные мемы,https://t.me/netolerantmemes,2890,-90,\n\t\t\t\t\t\t\t\t\t\t\t\tмы за равные права д...,https://t.me/dirtycringe
4,bruh.mp4,https://t.me/bruhvideos,2837,-40,\n\t\t\t\t\t\t\t\t\t\t\t\tShitty vidеоs. Barel...,https://t.me/plutocargo


In [33]:

                        
def test(min_sub, max_sub, pages=1,category = 'Юмор'):
    base_url = 'https://telemetr.me/'
    res=[]
    
    for p in range(1,pages + 1):
        response = session.get(f'{base_url}channels/cat/{category}/?order_column=participants_count&order_direction=DESC&page={p}&participants_from={min_sub}&participants_to={max_sub}&lang_ru=1&lang_uz=0&channel_type=opened&moderate=all&verified=all&detailed_bot_added=all&lang_code=ru')
        #'https://telemetr.me/channels/cat/Юмор/?about=%27%40%27&participants_from=300&participants_to=3000&channel_type=opened&lang_code=ru&detailed_bot_added=all'
        soup = BeautifulSoup(response.text)
        tables = soup.find_all('tr', class_ = ['tr_even','tr_odd'])

        for table in tables:
            #time.sleep(0.2)
            
            try:
                title = table.find('a', 'kt-ch-title').text
            except:
                continue
                
            
            channel_link = base_url + table.find('a', 'kt-ch-title').get('href')[1:]
            channel_info = session.get(channel_link)
            channel_info.encoding = 'utf-8' 

            soup_info = BeautifulSoup(channel_info.text)
            
            
            
            #br = soup_info.find('div', 'kt-widget__desc t_long').find_all('br')
            #print(br)
            #for b in br:
            #    a = b.find_all('a')
            #    for i in a:
            #        print(i.text)
            
            try:
                a = soup_info.find('div', 'kt-widget__desc t_long').find_all('a')
                admins = []
                for link in a:
                    admin = link.get('href')
                    if re.search('^https://t\.me/(?!\+)(?!.*[bB][oO][tT]$).*', admin):
                        admins.append(admin)
                if len(admins) == 0:
                    continue
            except:
                continue
            
                
            res.append({'title': title, 'admins': '\n'.join(admins)})
            
    res_df = pd.DataFrame(res)
    return res_df

results = test(300,3000,pages = 1)
results.to_excel('results.xlsx', sheet_name='Admins List', index=False)


In [72]:
protected_url = "https://telemetr.me/channels/cat/Юмор/?order_column=participants_count&order_direction=DESC&page=2&participants_from=500&participants_to=3000&lang_ru=1&lang_uz=0&channel_type=all&moderate=all&verified=all&detailed_bot_added=all&lang_code=any"
response = session.get(protected_url)
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup)

<!DOCTYPE html>
 <html lang="en"> <head> <meta charset="utf-8"/> <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/> <link href="/favicons/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/> <link href="/favicons/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/> <link href="/favicons/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/> <link href="/favicons/apple-touch-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/> <link href="/favicons/apple-touch-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/> <link href="/favicons/apple-touch-icon-120x120.png" rel="apple-touch-icon" sizes="120x120"/> <link href="/favicons/apple-touch-icon-144x144.png" rel="apple-touch-icon" sizes="144x144"/> <link href="/favicons/apple-touch-icon-152x152.png" rel="apple-touch-icon" sizes="152x152"/> <link href="/favicons/apple-touch-icon-180x180.png" rel="apple-touch-icon" sizes="180x180"/> <link href="/favicons/andr